In [1]:
import os, glob
import numpy as np
import pandas as pd
import sys
sys.path.insert(0, "../src")
import auxilary_functions as f
from generation_algorithm import *
import subprocess
import csv
import matplotlib.pyplot as plt
import json

cfg_file = "../src/config-ecoli.json"

sizes = ['500', '750', '1000']
os.chdir('../networks/')

In [2]:
def load_ffl_based_component():
    with open(cfg_file, 'r') as j:
        config_file = json.loads(j.read())
            
        interaction_matrix = f.get_interaction_matrix(config_file)
        motifs, counter = f.motif_search(config_file, interaction_matrix, batch_size=10000)
        motifs_orig = motifs["030T"]

        ffl_nodes = list(set(sum([list(map(int, x.split("_"))) for x in motifs_orig], [])))
        interaction_matrix_ffl = np.zeros((len(ffl_nodes), len(ffl_nodes)))
        for motif in motifs_orig:
            motif = f.split_motif(motif)
            motif_new = list(ffl_nodes.index(x) for x in motif)
            interaction_matrix_ffl[np.ix_(motif_new, motif_new)] = \
            interaction_matrix[np.ix_(motif, motif)]
        interaction_matrix_ffl.shape, interaction_matrix_ffl.sum()

        # Vertex-based motif network on FFL
        motifs_network = f.build_vmn(motifs_orig, verbose=True)
        V = nx.Graph(motifs_network)
        nx.is_connected(V)
        return interaction_matrix, motifs_orig, motifs_network, interaction_matrix_ffl

In [3]:
substrate_matrix1 = get_network_nucleus(
        yeast_matrix, ffl_motif, ffl_component, min_size=7, random_seed=19
    )
substrate_matrix2 = get_network_nucleus(
        yeast_matrix, ffl_motif, ffl_component, min_size=7, random_seed=30
    )

NameError: name 'yeast_matrix' is not defined

In [147]:
substrate_matrix1

array([[0., 0., 1., 0., 0., 0., 0., 0.],
       [1., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 1., 0., 0., 0., 0.],
       [0., 0., 1., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0., 0., 0., 0.]])

In [6]:
substrate_matrix2 = np.array([[0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [1., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0., 0., 0.]])

In [9]:
substrate_matrix2.sum()

9.0

In [150]:
full_substrate = np.zeros(shape=(substrate_matrix1.shape[0]+substrate_matrix2.shape[0],substrate_matrix1.shape[0]+substrate_matrix2.shape[0]))

In [151]:
full_substrate[0:substrate_matrix1.shape[0], 0:substrate_matrix1.shape[0]] = substrate_matrix1
full_substrate[substrate_matrix1.shape[0]:substrate_matrix1.shape[0]+substrate_matrix2.shape[0], substrate_matrix1.shape[0]:substrate_matrix1.shape[0]+substrate_matrix2.shape[0]] = substrate_matrix2

In [152]:
full_substrate

array([[0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0.

In [10]:
in_degrees = {'both1': 1, 'both2': 2, 'only_x': 100}
out_degrees = {'both1': 10, 'both2': 20, 'only_y': 200}

all_degrees = {k: in_degrees.get(k, 0) + out_degrees.get(k, 0) for k in set(in_degrees) | set(out_degrees)}

In [11]:
all_degrees

{'only_x': 100, 'only_y': 200, 'both1': 11, 'both2': 22}

## Convert motifs to FFLs // test

In [3]:
df = f.read_ecoli_network("../networks/stability_analysis_experiment/ecoli/fflatt/250/fflatt_transcriptional_network_0_nodes_250.tsv")
edges = df.astype(str)
edges.columns = ["tf", "tg"]
nodes = sorted(np.unique(np.concatenate((edges.tf.unique(), edges.tg.unique()))))
nodes = pd.DataFrame(data=range(len(nodes)), index=nodes, columns=["idx"])
edges_ = edges.join(nodes, on="tf").join(nodes, on="tg", lsuffix="_tf", rsuffix="_tg")
np_edges = edges_[["idx_tg", "idx_tf"]].values
interaction_matrix_test = f.build_Tnet(np_edges, len(nodes))

In [4]:
interaction_matrix_test

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [32]:
def filter_for_two_edge_motif(substrate_matrix, config_file):
    matrix_motifs, motifs_stats = f.motif_search(config_file, substrate_matrix, batch_size=10000)
    original_sum = substrate_matrix.sum()
    
    #iterate over all 3-node loops in network
    #!Note
    #deletion of 021C leads to 
    #for motif_type in [matrix_motifs['021U'], matrix_motifs['021C'], matrix_motifs['021D']]:
    for motif_type in [matrix_motifs['021C']]:
        inner_counts = 0
        print(motif_type)
        for motif in motif_type:
            print(motif)
            #iterate over all nodes in motif
            nodes = motif.split('_')
            out_degrees = {}
            in_degrees = {}
            for node in nodes:
                #calculate out-degree
                out_degrees[node] = substrate_matrix[int(node),:].sum()
                in_degrees[node] = substrate_matrix[:,int(node)].sum()
                
            #sort by out-degree
            out_degrees = {k: v for k, v in sorted(out_degrees.items(), key=lambda item: item[1])}
            in_degrees = {k: v for k, v in sorted(in_degrees.items(), key=lambda item: item[1])}
            
            nodes_out = list(out_degrees.keys())
            nodes_in = list(in_degrees.keys())
            
            all_degrees = {k: in_degrees.get(k, 0) + out_degrees.get(k, 0) for k in set(in_degrees) | set(out_degrees)}
            all_degrees = {k: v for k, v in sorted(all_degrees.items(), key=lambda item: item[1])}
            
            nodes_all = list(all_degrees.keys())
            
            #convert 021U to FFL               
            substrate_matrix[int(nodes_all[1]),int(nodes_all[2])] = 1
            inner_counts +=1
            
            #substrate_matrix[int(nodes_out[0]),int(nodes_out[2])] = 0
            
            #counter // do sum of the matrix instead to calculate the number of added edges
            
        print(f"number_of_motif_substitutions: {inner_counts}")
        
    for motif_type in [matrix_motifs['021D']]:
        inner_counts = 0
        print(motif_type)
        for motif in motif_type:
            #print(motif)
            #iterate over all nodes in motif
            nodes = motif.split('_')
            out_degrees = {}
            in_degrees = {}
            for node in nodes:
                #calculate out-degree
                out_degrees[node] = substrate_matrix[int(node),:].sum()
                in_degrees[node] = substrate_matrix[:,int(node)].sum()
                
            #sort by out-degree
            out_degrees = {k: v for k, v in sorted(out_degrees.items(), key=lambda item: item[1])}
            in_degrees = {k: v for k, v in sorted(in_degrees.items(), key=lambda item: item[1])}
            
            nodes_out = list(out_degrees.keys())
            nodes_in = list(in_degrees.keys())
            
            all_degrees = {k: in_degrees.get(k, 0) + out_degrees.get(k, 0) for k in set(in_degrees) | set(out_degrees)}
            all_degrees = {k: v for k, v in sorted(all_degrees.items(), key=lambda item: item[1])}
            
            nodes_all = list(all_degrees.keys())
            
            #convert 021U to FFL
            substrate_matrix[int(nodes_in[1]),int(nodes_in[2])] = 1
            inner_counts +=1
            
            #substrate_matrix[int(nodes_out[0]),int(nodes_out[2])] = 0
            
            #counter // do sum of the matrix instead to calculate the number of added edges
            
        print(f"number_of_motif_substitutions: {inner_counts}")
        
    loops_edges = substrate_matrix.sum() - original_sum
        
    return loops_edges, substrate_matrix

In [33]:
cfg = f.get_actual_parametrization("../src/config.json")
smth, actual_matrix = filter_for_two_edge_motif(interaction_matrix_test, cfg)

['14_109_174', '3_191_240', '114_190_229', '14_43_91', '25_91_214', '3_169_217', '147_158_185', '25_123_169', '4_169_207', '0_24_185', '25_136_216', '25_91_249', '45_114_136', '66_102_185', '3_185_233', '0_36_171', '0_137_169', '114_147_150', '14_105_174', '70_91_169', '91_106_169', '114_136_182', '0_110_185', '185_223_229', '207_229_244', '3_14_191', '0_3_230', '25_187_207', '0_69_226', '0_6_114', '55_114_207', '2_58_240', '25_177_207', '136_207_220', '0_49_196', '2_112_174', '102_174_247', '73_126_240', '25_29_174', '69_174_217', '25_140_169', '37_91_174', '0_174_208', '0_64_218', '25_60_174', '0_3_47', '136_174_181', '2_45_240', '43_91_185', '196_203_240', '0_27_242', '0_25_200', '25_93_207', '25_45_136', '14_25_188', '0_91_215', '2_192_207', '61_185_229', '69_217_240', '0_19_174', '14_53_207', '102_185_247', '14_25_187', '104_174_229', '25_196_203', '36_171_216', '114_169_173', '34_36_174', '107_114_174', '2_207_221', '0_108_216', '91_103_169', '2_196_204', '43_91_114', '33_91_169'

number_of_motif_substitutions: 11109


In [34]:
matrix_motifs, motifs_stats = f.motif_search(cfg, actual_matrix, batch_size=10000)

In [35]:
motifs_stats

{'021C': 1228, '021D': 11109, '021U': 30, '030C': 1, '030T': 714}

In [21]:
motifs_stats

{'021C': 1206, '021D': 11076, '021U': 32, '030C': 1, '030T': 700}